In [1]:
!pip install emoji
!pip install soynlp
!pip install --upgrade nomic

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 416.8/416.8 kB 10.2 MB/s eta 0:00:0000:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.2/40.2 kB 2.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Obtaining dependency information for jsonlines from https://files.pythonhosted.org/packages/f8/62/d9ba6323b9202dd2fe166beab8a86d29465c41a0288cbe229fac60c1ab8d/jsonlines-4.0.0-py3-none-any.whl.metadata
  Created wheel for nomic: filename=nomic-3.0.3-py3-none-any.whl size=40999 sha256=1921969c9432a0ad6dcc0fc2c257839bca7917e3819a7d614910b83def567580
  Stored in directory: /root/.cache/pip/wheels/75/dc/63/d22dea87e1b53bd0ca3afba90432966c9ace2f773e8dcb5d5f
Successfully built nomic


In [2]:
import nomic
import getpass

# API KEY Issue link : https://atlas.nomic.ai/cli-login
YOUR_API_KEY =  getpass.getpass("Enter your Nomic API Key: ")
nomic.login(YOUR_API_KEY)

Enter your Nomic API Key:  ·············································


In [6]:
import pandas as pd 

df = pd.read_excel('/kaggle/input/korean-hate-speech-sum/filtered_data.xlsx')
df.head()

,text,class
0,....한국적인 미인의 대표적인 분...너무나 곱고아름다운모습...그모습뒤의 슬픔을...,0
1,"1,2화 어설펐는데 3,4화 지나서부터는 갈수록 너무 재밌던데",0
2,10+8 진짜 이승기랑 비교된다,0
3,10년뒤 윤서인은 분명히 재평가될것임. 말하나하나가 틀린게없음,0
4,10살 차이가 넘을텐데 부부라고? 무슨 내용인지 긍금하네..,0


In [7]:
import re
import emoji
from soynlp.normalizer import repeat_normalize

def text_preprocess(text):
    pattern = re.compile(f'[^ .,?!/@$%~％·∼()\x00-\x7Fㄱ-ㅣ가-힣]+')
    url_pattern = re.compile(
        r'https?:\/\/(www\.)?[-a-zA-Z0-9@:%._\+~#=]{1,256}\.[a-zA-Z0-9()]{1,6}\b([-a-zA-Z0-9()@:%_\+.~#?&//=]*)')
 
    text = pattern.sub(' ', text)
    text = emoji.replace_emoji(text, replace='') #emoji 삭제
    text = url_pattern.sub('', text)
    text = text.strip()
    text = repeat_normalize(text, num_repeats=2)
    
    return text

df['text'] = df['text'].apply(lambda x: text_preprocess(x))
df.head()

,text,class
0,....한국적인 미인의 대표적인 분...너무나 곱고아름다운모습...그모습뒤의 슬픔을...,0
1,"1,2화 어설펐는데 3,4화 지나서부터는 갈수록 너무 재밌던데",0
2,10+8 진짜 이승기랑 비교된다,0
3,10년뒤 윤서인은 분명히 재평가될것임. 말하나하나가 틀린게없음,0
4,10살 차이가 넘을텐데 부부라고? 무슨 내용인지 긍금하네..,0


In [8]:
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification

model_name = "beomi/KcELECTRA-small-v2022"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=11, output_hidden_states=True)

# set device
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
model.to(device)

Some weights of ElectraForSequenceClassification were not initialized from the model checkpoint at beomi/KcELECTRA-small-v2022 and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.weight', 'classifier.out_proj.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


ElectraForSequenceClassification(
  (electra): ElectraModel(
    (embeddings): ElectraEmbeddings(
      (word_embeddings): Embedding(54343, 128, padding_idx=0)
      (position_embeddings): Embedding(512, 128)
      (token_type_embeddings): Embedding(2, 128)
      (LayerNorm): LayerNorm((128,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (embeddings_project): Linear(in_features=128, out_features=256, bias=True)
    (encoder): ElectraEncoder(
      (layer): ModuleList(
        (0-11): 12 x ElectraLayer(
          (attention): ElectraAttention(
            (self): ElectraSelfAttention(
              (query): Linear(in_features=256, out_features=256, bias=True)
              (key): Linear(in_features=256, out_features=256, bias=True)
              (value): Linear(in_features=256, out_features=256, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): ElectraSelfOutput(
              (dense): Li

In [10]:
model.load_state_dict(torch.load('/kaggle/input/sentence-embedding-electra/sentence_embedding.pt'))
model.eval()

ElectraForSequenceClassification(
  (electra): ElectraModel(
    (embeddings): ElectraEmbeddings(
      (word_embeddings): Embedding(54343, 128, padding_idx=0)
      (position_embeddings): Embedding(512, 128)
      (token_type_embeddings): Embedding(2, 128)
      (LayerNorm): LayerNorm((128,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (embeddings_project): Linear(in_features=128, out_features=256, bias=True)
    (encoder): ElectraEncoder(
      (layer): ModuleList(
        (0-11): 12 x ElectraLayer(
          (attention): ElectraAttention(
            (self): ElectraSelfAttention(
              (query): Linear(in_features=256, out_features=256, bias=True)
              (key): Linear(in_features=256, out_features=256, bias=True)
              (value): Linear(in_features=256, out_features=256, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): ElectraSelfOutput(
              (dense): Li

In [12]:
from torch.utils.data import DataLoader, Dataset

batch_size =32

# Custom Dataset 클래스
class MyDataset(Dataset):
    def __init__(self, dataframe):
        self.dataframe = dataframe

    def __len__(self):
        return len(self.dataframe)

    def __getitem__(self, idx):
        text = self.dataframe.iloc[idx, 0]
        label = self.dataframe.iloc[idx, 1]
        return text, label

my_dataset = MyDataset(df)
train_dataloader = DataLoader(my_dataset, shuffle=False, batch_size=batch_size)

In [13]:
# 임베딩을 저장할 리스트 초기화
embeddings = []

train_dataloader = DataLoader(my_dataset, shuffle=False, batch_size=batch_size)
# 데이터 로더를 통해 배치 단위로 처리
for texts, labgel in train_dataloader:
    inputs = tokenizer(texts, padding=True, return_tensors='pt')
    inputs = {k: v.to(device) for k,v in inputs.items()}
    with torch.no_grad():
        outputs = model(**inputs)
        cls_hidden_state = outputs.hidden_states[-1].cpu()
        mean_embedding = torch.mean(cls_hidden_state,dim=1).squeeze()
        embeddings.extend(mean_embedding)

In [16]:
# 일반적 
# 0. 일반발언 common 
# 1. 성차별 sex
# 2. 인종차별 
# 3. 종교차별 religion
# 4. 연령차별 age
# 5. 정치차별 political
# 6. 혐오욕설 
# 7. 출신차별
# 8. 외모차별
# 9. 성소수자 차별
# 10. 기타혐오 other 
class_dictionary = {
    0 : "normal" , 1: "sex", 2: "race", 3: "religion", 4:"age", 
    5: "political", 6: "abuse", 7 : "origin", 8: "appearance", 
    9: "sexual minority", 10:"other"
}

df['embbeded_sentence'] = embeddings
df['class'] = df['class'].apply(lambda x : class_dictionary[x])
df.head()

,text,class,embbeded_sentence
0,....한국적인 미인의 대표적인 분...너무나 곱고아름다운모습...그모습뒤의 슬픔을...,normal,"[tensor(0.9999), tensor(0.5262), tensor(-1.154..."
1,"1,2화 어설펐는데 3,4화 지나서부터는 갈수록 너무 재밌던데",normal,"[tensor(0.9422), tensor(0.5321), tensor(-1.196..."
2,10+8 진짜 이승기랑 비교된다,normal,"[tensor(0.8560), tensor(0.2837), tensor(-0.699..."
3,10년뒤 윤서인은 분명히 재평가될것임. 말하나하나가 틀린게없음,normal,"[tensor(1.1205), tensor(0.2194), tensor(-0.887..."
4,10살 차이가 넘을텐데 부부라고? 무슨 내용인지 긍금하네..,normal,"[tensor(-0.1273), tensor(0.2268), tensor(-0.37..."


In [18]:
import numpy as np
size = len(df)

matrix = np.zeros((size, 256))
class_list = df['class']

for idx, vectors in enumerate(df['embbeded_sentence']):
    vector = np.array(vectors)
    matrix[idx] =vector 

In [19]:
from sklearn.manifold import TSNE

tsne = TSNE(n_components=2, verbose=1, perplexity=40, n_iter=300)
vectors_tsne = tsne.fit_transform(matrix)
vectors_tsne[:10]

[t-SNE] Computing 121 nearest neighbors...
[t-SNE] Indexed 153719 samples in 0.028s...
[t-SNE] Computed neighbors for 153719 samples in 146.621s...
[t-SNE] Computed conditional probabilities for sample 1000 / 153719
[t-SNE] Computed conditional probabilities for sample 2000 / 153719
[t-SNE] Computed conditional probabilities for sample 3000 / 153719
[t-SNE] Computed conditional probabilities for sample 4000 / 153719
[t-SNE] Computed conditional probabilities for sample 5000 / 153719
[t-SNE] Computed conditional probabilities for sample 6000 / 153719
[t-SNE] Computed conditional probabilities for sample 7000 / 153719
[t-SNE] Computed conditional probabilities for sample 8000 / 153719
[t-SNE] Computed conditional probabilities for sample 9000 / 153719
[t-SNE] Computed conditional probabilities for sample 10000 / 153719
[t-SNE] Computed conditional probabilities for sample 11000 / 153719
[t-SNE] Computed conditional probabilities for sample 12000 / 153719
[t-SNE] Computed conditional prob

array([[ -8.102011  ,   4.329313  ],
       [-11.280683  ,   0.94274604],
       [ -2.9872715 ,  -2.1627765 ],
       [ -6.169154  ,   1.434594  ],
       [ -1.8016375 ,  -8.763573  ],
       [-11.769028  ,  -0.5025457 ],
       [ -1.5053109 ,  -8.937902  ],
       [ -4.9061766 ,  -0.71799934],
       [ -5.4378276 ,  -7.4274774 ],
       [ -2.3602536 ,  -3.122212  ]], dtype=float32)

In [22]:
dataset = []

for idx in range(len(df)):
    text = df['text'][idx]
    label = df['class'][idx]
    dataset.append({
        'text' : text, 'id': idx, 'label': label
    })

In [24]:
from nomic import atlas

# Create Atlas project
project = atlas.map_data(
    data= dataset,
    embeddings=vectors_tsne,
    embedding_model=None,
    projection=None,
    identifier='label',
    id_field='id'
  )

print(project)

2024-01-16 14:04:42.648 | INFO     | nomic.dataset:_create_project:842 - Creating dataset `label`
2024-01-16 14:04:42.761 | INFO     | nomic.atlas:map_data:98 - Uploading data to Atlas.
2024-01-16 14:04:42.945 | WARNING  | nomic.dataset:_validate_and_correct_arrow_upload:338 - id_field is not a string. Converting to string from int32
31it [00:04,  7.23it/s]                        
2024-01-16 14:04:47.245 | INFO     | nomic.dataset:_add_data:1510 - Upload succeeded.
2024-01-16 14:04:47.248 | INFO     | nomic.atlas:map_data:113 - `tigere0723/label`: Data upload succeeded to dataset`
2024-01-16 14:04:47.350 | WARNING  | nomic.dataset:create_index:1095 - You did not specify the `topic_label_field` option in your topic_model, your dataset will not contain auto-labeled topics.
2024-01-16 14:04:47.888 | INFO     | nomic.dataset:create_index:1219 - Created map `label` in dataset `tigere0723/label`: https://atlas.nomic.ai/map/cee385e3-8b73-4a38-919f-c3ffd62b74af/e1d71557-8d05-493d-a3ef-fcdfb90e

label: https://atlas.nomic.ai/map/cee385e3-8b73-4a38-919f-c3ffd62b74af/e1d71557-8d05-493d-a3ef-fcdfb90e411e
